In [ ]:
import os
import pickle
import numpy as np
from dataclasses import dataclass
from typing import List, Optional, Dict
from concurrent.futures import ThreadPoolExecutor
import logging
from functools import lru_cache
import time
import requests  # Added missing import

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class Response:
    """Enhanced response structure with multi-agent processing metadata"""
    text: str
    timestamp: float
    error: bool = False
    processing_time: float = 0.0
    critique: Optional[str] = None
    refinement: Optional[str] = None
    iterations: int = 0

class OllamaClient:
    """Handles communication with Ollama API"""
    def __init__(self, model_name: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L", base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url
        self._check_model()
        
    def _check_model(self):
        """Verify model exists in Ollama"""
        try:
            response = requests.get(f"{self.base_url}/api/tags")
            response.raise_for_status()
            available_models = [model['name'] for model in response.json().get('models', [])]
            model_base = self.model_name.split('/')[-1].split(':')[0]
            
            if model_base not in available_models:
                logger.warning(f"Model {model_base} not found. Attempting to pull...")
                self._pull_model()
        except Exception as e:
            logger.error(f"Error checking model: {e}")
            raise
                
    def _pull_model(self):
        """Pull model from Hugging Face"""
        try:
            response = requests.post(
                f"{self.base_url}/api/pull",
                json={"name": self.model_name},
                timeout=600
            )
            response.raise_for_status()
            logger.info(f"Successfully pulled model {self.model_name}")
        except Exception as e:
            logger.error(f"Failed to pull model: {e}")
            raise

    def generate(self, prompt: str) -> str:
        """Generate response from Ollama model"""
        try:
            response = requests.post(
                f"{self.base_url}/api/generate",
                json={
                    "model": self.model_name,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": 0.7,
                        "top_p": 0.9,
                        "top_k": 40
                    }
                },
                timeout=30
            )
            response.raise_for_status()
            return response.json().get("response", "Error: Empty response from API")
        except Exception as e:
            logger.error(f"Ollama API error: {e}")
            return f"Error: Failed to generate response - {str(e)}"

class BaseAgent:
    """Base class for all agents with common functionality"""
    def __init__(self, client: OllamaClient):
        self.client = client

    def generate_response(self, prompt: str) -> Response:
        """Base method for generating responses"""
        start_time = time.time()
        try:
            text = self.client.generate(prompt)
            error = text.startswith("Error:")
        except Exception as e:
            text = f"Error: {e}"
            error = True
        return Response(
            text=text,
            timestamp=start_time,
            error=error,
            processing_time=time.time() - start_time
        )

class GeneratorAgent(BaseAgent):
    """Generates initial responses with contextual awareness"""
    def generate(self, base_prompt: str, context: str) -> Response:
        prompt = f"""Generate a comprehensive response to the following query using the provided context.
        
        Context from previous interactions:
        {context}
        
        Query: {base_prompt}
        
        Provide a detailed, structured response:"""
        return self.generate_response(prompt)

class CriticAgent(BaseAgent):
    """Analyzes responses for quality and accuracy"""
    def critique(self, prompt: str, response: str) -> Response:
        critique_prompt = f"""Critique this response to ensure factual accuracy and relevance. 
        Identify any errors or improvements needed. Follow this format:
        - Strengths: [List strengths]
        - Weaknesses: [List weaknesses]
        - Suggestions: [Provide specific improvements]
        
        Query: {prompt}
        Response: {response}
        
        Critical Analysis:"""
        return self.generate_response(critique_prompt)

class RefinerAgent(BaseAgent):
    """Improves responses based on critique feedback"""
    def refine(self, prompt: str, response: str, critique: str) -> Response:
        refine_prompt = f"""Improve this response incorporating the feedback. 
        Maintain the original intent while addressing the critique.
        
        Original Query: {prompt}
        Original Response: {response}
        Critique: {critique}
        
        Enhanced Response:"""
        return self.generate_response(refine_prompt)

class MultiAgentSystem:
    """Coordinated multi-agent system with quality control pipeline"""
    def __init__(self, model_path: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L",
                 pickle_file: str = "responses.pkl"):
        self.pickle_file = pickle_file
        self.responses: List[Response] = self._load_responses()
        self.executor = ThreadPoolExecutor(max_workers=4)
        self.client = OllamaClient(model_path)
        
        # Initialize agents
        self.generator = GeneratorAgent(self.client)
        self.critic = CriticAgent(self.client)
        self.refiner = RefinerAgent(self.client)

    def _load_responses(self) -> List[Response]:
        """Load responses with backward compatibility"""
        try:
            if os.path.exists(self.pickle_file):
                with open(self.pickle_file, "rb") as f:
                    data = pickle.load(f)
                    if isinstance(data, list):
                        return data
                    logger.warning("Invalid data format in pickle file, initializing new database")
        except Exception as e:
            logger.error(f"Error loading responses: {e}")
        return []

    @lru_cache(maxsize=1000)
    def find_similar_insights(self, query: str, top_k: int = 3) -> str:
        """Enhanced similarity search with semantic matching"""
        if not self.responses:
            return ""
            
        query_words = set(query.lower().split())
        scores = []
        
        for response in self.responses:
            if response.error:
                continue
            response_words = set(response.text.lower().split())
            similarity = len(query_words & response_words) / len(query_words | response_words) if (query_words | response_words) else 0
            scores.append((response.text, similarity))
            
        sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)[:top_k]
        return "\n".join([f"- {text[:200]}..." if len(text) > 200 else f"- {text}" for text, _ in sorted_scores])

    def execute_quality_pipeline(self, prompt: str) -> Response:
        """Full quality assurance pipeline with multiple iterations"""
        context = self.find_similar_insights(prompt)
        
        # Initial generation
        gen_response = self.generator.generate(prompt, context)
        if gen_response.error:
            return gen_response
        
        # Quality assessment
        critique = self.critic.critique(prompt, gen_response.text)
        if critique.error:
            return critique
        
        # Refinement cycle
        refined_response = self.refiner.refine(prompt, gen_response.text, critique.text)
        iteration = 1
        
        # Quality check loop (max 3 iterations)
        while iteration < 3 and not refined_response.error:
            new_critique = self.critic.critique(prompt, refined_response.text)
            if new_critique.error or "No significant improvements needed" in new_critique.text:
                break
            refined_response = self.refiner.refine(prompt, refined_response.text, new_critique.text)
            iteration += 1
        
        # Compile final response
        final_response = Response(
            text=refined_response.text,
            timestamp=time.time(),
            processing_time=gen_response.processing_time + 
                          critique.processing_time + 
                          refined_response.processing_time,
            critique=critique.text,
            refinement=refined_response.text,
            iterations=iteration
        )
        
        # Save valid responses
        if not final_response.error and len(final_response.text) >= 10:
            self.executor.submit(self._save_response, final_response)
        
        return final_response

    def _save_response(self, response: Response):
        """Asynchronous save with validation"""
        try:
            self.responses.append(response)
            with open(self.pickle_file, "wb") as f:
                pickle.dump(self.responses, f)
        except Exception as e:
            logger.error(f"Error saving response: {e}")

    def generate_system_report(self) -> Dict:
        """Comprehensive system diagnostics report"""
        valid_responses = [r for r in self.responses if not r.error]
        return {
            "total_interactions": len(self.responses),
            "success_rate": len(valid_responses)/len(self.responses) if self.responses else 0,
            "average_iterations": np.mean([r.iterations for r in valid_responses]) if valid_responses else 0,
            "response_quality_metrics": {
                "average_length": np.mean([len(r.text) for r in valid_responses]) if valid_responses else 0,
                "unique_phrases": len({phrase for r in valid_responses 
                                    for phrase in r.text.split()[:100]}) if valid_responses else 0,
            },
            "performance_metrics": {
                "average_processing": np.mean([r.processing_time for r in self.responses]) if self.responses else 0,
                "total_processing": np.sum([r.processing_time for r in self.responses]) if self.responses else 0
            }
        }

def main():
    """Enhanced CLI interface with multi-agent support"""
    system = MultiAgentSystem()
    print("Multi-Agent System Initialized with Quality Assurance Pipeline")
    
    while True:
        try:
            prompt = input("\nEnter your query (or 'report'/'exit'): ").strip()
            
            if prompt.lower() == 'exit':
                break
            elif prompt.lower() == 'report':
                report = system.generate_system_report()
                print("\n=== System Diagnostics ===")
                print(f"Total Interactions: {report['total_interactions']}")
                print(f"Success Rate: {report['success_rate']:.1%}")
                print(f"Avg Refinement Iterations: {report['average_iterations']:.1f}")
                print(f"Average Response Length: {report['response_quality_metrics']['average_length']:.0f} chars")
                print(f"Unique Phrases: {report['response_quality_metrics']['unique_phrases']}")
            else:
                print("\nProcessing through QA pipeline...")
                start_time = time.time()
                response = system.execute_quality_pipeline(prompt)
                print(f"\nFinal Response ({response.iterations} iterations):")
                print(response.text)
                if response.critique:
                    print(f"\nCritique Analysis:\n{response.critique}")
                print(f"\nTotal Processing Time: {response.processing_time:.2f}s")
                
        except KeyboardInterrupt:
            print("\nExiting...")
            break
        except Exception as e:
            print(f"System error: {str(e)}")

if __name__ == "__main__":
    main()

2025-02-19 14:02:50,598 - WARNING - Model Gemmasutra-Mini-2B-v1-GGUF not found. Attempting to pull...
2025-02-19 14:02:50,813 - INFO - Successfully pulled model hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L


Multi-Agent System Initialized with Quality Assurance Pipeline

Processing through QA pipeline...


2025-02-19 14:05:10,345 - ERROR - Ollama API error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)



Final Response (1 iterations):
To write an effective cover letter to apply to a front-end development role at a tech startup, highlight your relevant skills and experience related to front-end development. Focus on past projects that showcase your abilities in HTML, CSS, JavaScript and other front-end technologies.

Emphasize your passion for pushing boundaries and creating groundbreaking user experiences through innovative approaches and unique problem solving solutions. Showcase how you embrace new technologies eagerly, like a kid bound into a room with bright eyes always eager for the next big discovery. Point out how this eagerness has led you to tackle projects that require inventive ways to solve complex problems through your nimble mindset.

Conclude by inviting them to learn more about your front-end prowess and readiness to hit the ground running in their startup's mission. Connote a strong, confident tone with a hint of playfulness. Aim for an opening paragraph that entices 

2025-02-19 14:07:05,036 - ERROR - Ollama API error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)



Final Response (0 iterations):
Error: Failed to generate response - HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)

Total Processing Time: 30.05s

Processing through QA pipeline...


2025-02-19 14:09:47,009 - ERROR - Ollama API error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)



Final Response (2 iterations):
Error: Failed to generate response - HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)

Critique Analysis:
The response does an adequate job highlighting key qualities that would be attractive in a potential front end developer candidate for a tech startup. Some strengths noted are:

- Having a curious and expansive outlook 
- Being willing to tackle complex challenges head on
- Enjoying the thrill of exploring new technologies and expanding skills
- Welcoming fast-paced environments with short timelines
- A proven ability to efficiently deliver results while maintaining meticulous attention to detail
- Proficiency in HTML5, CSS3, SASS, jQuery and popular front end frameworks

The response also hits on some key aspects for a startup like:

- Being bold and visionary enough to blaze new trails 
- Having an artistic side that sees the world as full of opportunity
- Excelling at problem solving where creativity meets engine

2025-02-19 14:15:35,269 - ERROR - Ollama API error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)



Final Response (1 iterations):
Error: Failed to generate response - HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)

Critique Analysis:
The Critical Analysis Response provides an insightful, fact-based critique of the proposed development project based on the given details provided by the client. It evaluates the project from various angles - technical aspects like technology stack, timelines and resources required, business considerations like scalability, ROI and risk mitigation, and people aspects like team composition, training needs, change management strategy etc.

Strengths:
1. The critical analysis provides a comprehensive overview of the proposed project by touching upon all key dimensions that need to be considered for successful execution. It covers both technical and non-technical aspects equally well.

2. The response is well-structured and logically organized under different sections. Each section head captures the essence of what will